In [ ]:
import pandas as pd
import numpy as np
import feather

import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Set to true for export of the training, validation and test set to be done
proceed_export = False
rdata_export = False
populate_type = ['train','val','test'][0]

# Import Data

In [ ]:
df = pd.read_csv('static_patient_data_v2.csv',
                 parse_dates = ['intime', 'outtime','hospital_deathtime', 'DOD','DOD_HOSP','DOD_SSN','admittime'],
                 dtype={
                     'days_live':'Int64'
                     ,'years_live': 'Int64'
                     }
                 ).drop(['intime_1'], axis = 1)

In [ ]:
df.describe()

In [ ]:
df.dtypes

# Identify Unique Patient Rows
Extracts the first icustay_id based upon entry

In [ ]:
patient_df = df[df['ICUSTAY_AGE_GROUP'] == 'adult'].sort_values(by='intime', ascending = True).drop_duplicates(subset='subject_id', keep='first')

patient_df.drop('ICUSTAY_AGE_GROUP', axis = 1, inplace = True)

print(len(patient_df))

36616


# Generate Time to Death and Flags on Anytime/Hospital/ICU location of death
Includes analysis for consistency, and removal of unneseary fields 

In [ ]:
patient_df['time_of_death'] = np.where(patient_df['hospital_deathtime'].notnull(), patient_df['hospital_deathtime'], patient_df['DOD'])

In [ ]:
patient_df['time_until_death']  = patient_df['time_of_death'] - patient_df['intime']

In [ ]:
# Excludes records with negative time until death. Around 70 records.
patient_df = patient_df[np.logical_not(patient_df['time_until_death'] < pd.Timedelta(-1,'s'))].sort_values(by='time_until_death', ascending = True)

In [ ]:
print(len(patient_df))

36550


In [ ]:
# 5 records when DOD is after the Hospital Deathtime in the time_of_death. The earlier death in time_of_death should be better.
# hospital_deathtime is sometimes right before DOD_SNN, and earlier time (which should be better) is stored in time_of_death
# Some inconstiencies where DOD_SNN is null when someone died in a hospital, but the hospital time of death is included in Time of Death
# patient_df[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

# Renaming fields as well as dropping fields that are no longer needed
patient_df = patient_df.drop(['DOD_SSN','DOD_HOSP','hospital_deathtime','DOD'], axis = 1).rename(columns = {
    'EXPIRE_FLAG': 'anytime_expire_flag'
    , 'ICUSTAY_EXPIRE_FLAG': 'icu_expire_flag'
    ,'ICUSTAY_AGE_GROUP': 'icustay_age_group'
    })

In [ ]:
patient_df = patient_df.sort_values(by='intime', ascending = True).reset_index().drop(['index'], axis = 1)

In [ ]:
# 89 records had a year difference from age at admission and age at ICU admission, so using the ICU admission age.
patient_df = patient_df.drop(['admit_age'], axis = 1)

In [ ]:
# Changing order of fields for easy use
patient_df = patient_df[['subject_id', 'hadm_id', 'icustay_id', 'time_of_death', 'time_until_death',
       'anytime_expire_flag', 'hospital_expire_flag', 'icu_expire_flag', 'admittime', 'intime',
       'gender', 'icu_admit_age']]

In [ ]:
patient_df.sort_values(by = 'time_until_death', ascending = True)

# Train/ Validate/ Test Split
Validate can be used to give us some statistics during the phase 1 report that we can use to inform our final models which can be evaluated on the test set

In [ ]:
from sklearn.model_selection import train_test_split

patient_train, patient_test = train_test_split(
patient_df, test_size=0.20, random_state=5, shuffle = True)

In [ ]:
patient_df[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

In [ ]:
patient_train[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

In [ ]:

# Confirm train split is done correctly
saved_train_distribution = patient_train[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

if saved_train_distribution[0].tolist() == [17879, 8232, 861, 2268]:
    print('Train Split Done Correctly')
else:
    raise ValueError('Split was done incorrectly, review')

Train Split Done Correctly


In [ ]:
patient_test[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

In [ ]:
# Split into a validation set

In [ ]:
patient_train, patient_val = train_test_split(
patient_train, test_size=0.20, random_state=14, shuffle = False)

In [ ]:
patient_train[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

In [ ]:
# Confirm second train and validation split is done correctly

# Check overall count for each class
saved_train_distribution = patient_train[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

if saved_train_distribution[0].tolist() == [14239, 6649, 696, 1808]:
    print('Second train split done correctly with proper distribution between classes')
else:
    raise ValueError('Split was done incorrectly, review')

# Check first 5 records are correct
if patient_train.head().index.tolist() == [34451, 8697, 31360, 19754, 36435]:
    print('Second train split done correctly as the first 5 records in the training set are expected')
else:
    raise ValueError('Split was done incorrectly, first 5 records are incorrect')

Second train split done correctly with proper distribution between classes
Second train split done correctly as the first 5 records in the training set are expected


In [ ]:
patient_val[['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']].groupby(by=['anytime_expire_flag','hospital_expire_flag','icu_expire_flag']).size().reset_index()

In [ ]:
print(len(patient_train), len(patient_val), len(patient_test))
total = len(patient_train) + len(patient_val) + len(patient_test)
print(len(patient_train)/total, len(patient_val)/total, len(patient_test)/total)
print(total)

# Populate Train, Val, or Test

In [ ]:
if populate_type == 'train':
  patient_split = patient_train
elif populate_type == 'val':
  patient_split = patient_val
elif populate_type == 'test':
  patient_split = patient_test
else:
  raise ValueError('Incorrect type of data used')

# Calculation of Time until enter ICU

In [ ]:
patient_split['admission_time_until_icu'] = patient_split['intime'] - patient_split['admittime']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
mask = (patient_split['admission_time_until_icu'] < pd.Timedelta(-1,'s'))

print('Total Records with proper admission time until ICU', len(patient_split[mask]) )
              
#patient_split[mask]
patient_split.loc[mask,'admission_time_until_icu'] = pd.Timedelta(0,'s')

Total Records with proper admission time until ICU 82


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
# Confirming no records remain
patient_split[patient_split['admission_time_until_icu'] < pd.Timedelta(-1,'s')]

,subject_id,hadm_id,icustay_id,time_of_death,time_until_death,anytime_expire_flag,hospital_expire_flag,icu_expire_flag,admittime,intime,gender,icu_admit_age,admission_time_until_icu


In [ ]:
patient_split['hours_until_icu_admission'] = patient_split['admission_time_until_icu'].astype('timedelta64[h]')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
patient_split.sort_values(by='hours_until_icu_admission', ascending = True).head()

In [ ]:
len(patient_split)

7310

In [ ]:
plt.hist(patient_split['admission_time_until_icu'].astype('timedelta64[h]'), 10000)
plt.xlim(0, 6)
plt.xlabel('Hours until Enter ICU')
plt.ylabel('Frequency')
plt.title('Arounud half of all ICU admissions are very fast, in the first hour')
plt.grid(True)

In [ ]:
plt.hist(patient_split['admission_time_until_icu'].dropna().astype('timedelta64[h]') , bins = 1000, cumulative=True, density=True, histtype='step',
                            label='Empirical')
plt.xlim(0, 72)
plt.xlabel('Hours admitted to hospital before ICU entry')
plt.ylabel('Cumulative Distribution')
plt.title('90% of all admission to ICU happen in the first 3 days of general admission to a hospital')
plt.grid(True)

plt.show()

# entire_mimiciii_clinical_icustays_table Features:
1. dbsource
2. first_careunit

In [ ]:
icustays = pd.read_csv('entire_mimic_iii_exported_tables/entire_mimiciii_clinical_icustays_table.csv',
                parse_dates = ['INTIME', 'OUTTIME']
                 )

In [ ]:
icustays.dtypes

In [ ]:
icustays.isnull().sum()

In [ ]:
icustays = icustays[['ICUSTAY_ID','DBSOURCE','FIRST_CAREUNIT','INTIME','OUTTIME']]
icustays = icustays.rename(columns = {'ICUSTAY_ID': 'icustay_id',
                                      'DBSOURCE': 'dbsource', 
                                    'FIRST_CAREUNIT': 'first_careunit'})
icustays.head(1)

In [ ]:
patient_split = patient_split.merge(icustays, on = 'icustay_id', how = 'left')

In [ ]:
patient_split.head()

In [ ]:
patient_split.groupby(by='dbsource').count()['subject_id']

In [ ]:
patient_split.groupby(by='first_careunit').count()['subject_id']
# SICU is Surgical Intensive Care Unit, TSICU is Trauma Surgical Intensive Care Unit, ect

# mimiciii_clinical.admissions_table Features:
1. ADMISSION_TYPE
2. DIAGNOSIS

In [ ]:
admissions = pd.read_csv('entire_mimic_iii_exported_tables/entire_mimiciii_clinical.admissions_table.csv',
                parse_dates = ['DISCHTIME']
                 )

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
admissions.dtypes

In [ ]:
admissions

In [ ]:
admissions.groupby(by='MARITAL_STATUS')['ROW_ID'].count()

In [ ]:
admissions.groupby(by='INSURANCE')['ROW_ID'].count()

In [ ]:
# #.join(pd.get_dummies(admissions[['ADMISSION_TYPE','MARITAL_STATUS','INSURANCE']], drop_first = False))
admissions = admissions[['DISCHTIME', 'HADM_ID','ADMISSION_TYPE','MARITAL_STATUS','INSURANCE']] \
  .rename(columns= {'HADM_ID': 'hadm_id'
  ,'ADMISSION_TYPE': 'admission_type'
  , 'MARITAL_STATUS': 'marital_status'
  , 'INSURANCE': 'insurance'
  , 'DISCHTIME': 'dischtime'}) 
  #.drop(['admission_type','marital_status','insurance','ADMISSION_TYPE_NEWBORN'], axis = 1)

In [ ]:
patient_split.merge(admissions, on = 'hadm_id', how = 'inner').rename(columns= {
    'OUTTIME': 'outtime'}).drop(['INTIME'], axis = 1)

In [ ]:
print(len(patient_split))
patient_split = patient_split.merge(admissions, on = 'hadm_id', how = 'inner').rename(columns= {
    'OUTTIME': 'outtime'}).drop(['INTIME'], axis = 1)

# Time from enter ICU to leave hospital (converted to days)
patient_split['time_until_hospital_exit'] = patient_split['dischtime'] - patient_split['intime'] # time from ICU entry to hospital exit
patient_split['days_until_hospital_exit'] = patient_split['time_until_hospital_exit'].astype('timedelta64[D]')

# cleaning one record with missing data
patient_split.loc[patient_split['time_until_hospital_exit'] < pd.Timedelta(0,'s'),'time_until_hospital_exit'] = pd.Timedelta(0,'s')
patient_split.loc[patient_split['days_until_hospital_exit'] < 0,'days_until_hospital_exit'] = 0

7310


# Merge in ICD Groups

In [ ]:
icd_data = pd.read_csv('diag.csv'
                #, parse_dates = ['DISCHTIME']
                 )

icd_data = icd_data[['HADM_ID'] + icd_data.columns.tolist()[-17:]].add_prefix('icd_').rename(columns= {'icd_HADM_ID': 'hadm_id'})

In [ ]:
patient_split = patient_split.merge(icd_data, on = 'hadm_id', how = 'left')

patient_split[icd_data.columns.tolist()[1:]] = patient_split[icd_data.columns.tolist()[1:]].fillna(value=0)

# Feature Selection
### Transform Features to Numeric prior to analysis

In [ ]:
patient_split#.dtypes

In [ ]:
# Time until death -> Hours until death

patient_split['hours_until_death'] = patient_split['time_until_death'].astype('timedelta64[h]')

patient_split['days_until_death'] = patient_split['time_until_death'].astype('timedelta64[D]')

In [ ]:
# Gender -> female boolean

patient_split['female'] = np.where(patient_split['gender']== 'F', 1, 0)
patient_split = patient_split.drop(['gender'], axis = 1)

In [ ]:
# first_careunit -> dummy variables (CCU	CSRU	MICU	SICU	TSICU)
# patient_split = patient_split.join(pd.get_dummies(patient_split["first_careunit"], drop_first = True))
# patient_split = patient_split.drop(['first_careunit'], axis = 1)
# http://data.patientcarelink.org/staffing/acronyms.cfm

## Set values for Living Patients

In [ ]:
# Hours until Death

max_time_until_death = patient_split['hours_until_death'].max()
mask = patient_split['hours_until_death'].isnull()
mask

In [ ]:
max_time_until_death

103969.0

In [ ]:
index_hours_until_death = patient_split.columns.tolist().index('hours_until_death')

In [ ]:
patient_split.iloc[mask[mask].index, index_hours_until_death]= max_time_until_death

# Merge in Validation Diagnosis

In [ ]:
diagnosis = pd.read_csv('static_' + populate_type + '_data_diag.csv',
                  parse_dates = ['dischtime']
                        )
print(diagnosis.dtypes[-21:])

subject_id_y               int64
diagnosis                 object
admission_type            object
dischtime         datetime64[ns]
cardio                     int64
infection                  int64
brain                      int64
mental                     int64
gastro                     int64
trauma                     int64
respiratory                int64
organ                      int64
blood                      int64
acute                      int64
failure                    int64
cancer                     int64
back                       int64
bone                       int64
pain                       int64
other                      int64
totalcats                  int64
dtype: object


In [ ]:
diagnosis = diagnosis.iloc[:,-21:].rename(columns={'subject_id_y':'subject_id'})

In [ ]:
patient_split = patient_split.merge(diagnosis, on = 'subject_id', how = 'inner')

In [ ]:
print(len(patient_split))

In [ ]:
patient_split = patient_split.rename(columns = {
    'dischtime_x': 'dischtime'
    ,'admission_type_x':'admission_type'}).drop(['admission_type_y','dischtime_y'], axis = 1)

# Calculate Length of Time in ICU

In [ ]:
patient_split

In [ ]:
patient_split['time_until_leave_ICU']  = patient_split['outtime'] - patient_split['intime']
patient_split['hours_until_leave_ICU'] = patient_split['time_until_leave_ICU'].astype('timedelta64[h]')

# cleaning one record with missing data
patient_split.loc[patient_split['time_until_leave_ICU'].isnull(),'time_until_leave_ICU'] = pd.Timedelta(0,'s')
patient_split.loc[patient_split['hours_until_leave_ICU'].isnull(),'hours_until_leave_ICU'] = 0

In [ ]:
patient_split['time_until_leave_ICU'].describe()

# What is the average time until an ICU patient leaves the hospital?
# Maybe predicting ICU patients and when they would leave ICU, can use patients that survive to get larger sample, but figure out factoring patients who die... (NEED TO EXCLUDE WITH MODEL..)

In [ ]:
plt.hist(patient_split['time_until_leave_ICU'].astype('timedelta64[D]'), 10)

plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age of Adult Patients is High, Clustering around 70')
plt.grid(True)

plt.show()

In [ ]:
len(set(patient_split['subject_id'].tolist()))

7310

# Time Enter ICU and Mock Statistics

In [ ]:
patient_split = patient_split.sort_values(by='intime', ascending = True)

In [ ]:
def convert_year_month_day(x):
  string_date = str(x.year) + '-' + str(x.month) + '-' + str(x.day)
  #print(string_date)
  #print(type(string_date))
  #return(pd.to_datetime(string_date, format='%Y%m%d', errors = 'ignore') )
  try:
    return(pd.to_datetime(string_date, format='%Y-%m-%d', errors = 'coerce'))
    #return(datetime.date(dt.year, dt.month, dt.day))
  except ValueError:
    return(np.NaN)

convert_year_month_day(pd.to_datetime('2012-1-1', format='%Y-%m-%d', errors = 'coerce'))

Timestamp('2012-01-01 00:00:00')

In [ ]:
patient_split['first_day_of_hospital'] = pd.to_datetime('20111130', format='%Y%m%d', errors='ignore')

# Generate list of number of days until enter ICU

new_patients_a_day = 3

num_rows = len(patient_split)
print('num rows:', num_rows)
num_admissions = int(num_rows/new_patients_a_day)
print('Number of admissions:', num_admissions)
#num_rows = 52

extra_rows =  num_rows%new_patients_a_day
print(extra_rows)
extra_row_list = extra_rows * [num_admissions + 1]

t = [new_patients_a_day * [i] for i in range(num_admissions) ] 

t.append(extra_row_list)

days_until_entry = []
for sublist in t:
    for item in sublist:
        days_until_entry.append(item)

print(len(days_until_entry), days_until_entry[-10:])

patient_split['TEST_days_until_icu_entry'] = days_until_entry

patient_split['TEST_date_diff_days_until_icu_entry'] = patient_split['TEST_days_until_icu_entry'].apply(lambda x: pd.Timedelta(x,'D'))

patient_split['TEST_date_enter_icu'] = patient_split['first_day_of_hospital'] + patient_split['TEST_date_diff_days_until_icu_entry']

num rows: 7310
Number of admissions: 2436
2
7310 [2433, 2433, 2434, 2434, 2434, 2435, 2435, 2435, 2437, 2437]


In [ ]:
patient_split['TEST_date_pass_away']  = patient_split['time_until_death'] + patient_split['TEST_date_enter_icu']
patient_split['TEST_date_pass_away'] = patient_split['TEST_date_pass_away'].apply(lambda x: convert_year_month_day(x))

patient_split['TEST_date_leave_icu']  = patient_split['time_until_leave_ICU'] + patient_split['TEST_date_enter_icu']
patient_split['TEST_date_leave_icu'] = patient_split['TEST_date_leave_icu'].apply(lambda x: convert_year_month_day(x))

patient_split['TEST_date_leave_hospital']  = patient_split['time_until_hospital_exit'] + patient_split['TEST_date_enter_icu']
patient_split['TEST_date_leave_hospital'] = patient_split['TEST_date_leave_hospital'].apply(lambda x: convert_year_month_day(x))

# Need to use hours of the day when someone enters the ICU...

In [ ]:
patient_split[patient_split['subject_id'] == 4246]

In [ ]:
index = pd.DataFrame(index=pd.date_range(patient_split.TEST_date_enter_icu.min(), patient_split.TEST_date_leave_icu.max()))

df1 = patient_split[patient_split['TEST_date_leave_icu'] < pd.to_datetime('20121231', format='%Y%m%d', errors='ignore')][['subject_id','TEST_date_enter_icu','TEST_date_leave_icu']]
index = pd.DataFrame(index=pd.date_range(pd.to_datetime('20111130', format='%Y%m%d', errors='ignore'), pd.to_datetime('20121231', format='%Y%m%d', errors='ignore')))
indices_of_time = index.index.tolist()

In [ ]:
patients_over_time = list()
for index_num, date in enumerate(index.index.tolist()):
  str_date = str(date)[0:10]
  #print(index_num, str_date)
  num_patients = len(patient_split[(patient_split['TEST_date_enter_icu'] <= indices_of_time[index_num]) & (patient_split['TEST_date_leave_icu'] >= indices_of_time[index_num]) ])
  patients_over_time.append(num_patients)

print(len(patients_over_time))

print(patients_over_time[31], indices_of_time[31])
print(patients_over_time[32], indices_of_time[32])
print(patients_over_time[39], indices_of_time[39])
print(patients_over_time[46], indices_of_time[46])

patients_over_time[-5:]
plt.rcParams["figure.figsize"]=10,5
plt.plot(indices_of_time[32:46], patients_over_time[32:46],)
plt.xlabel('Date')
plt.ylabel('Patients in ICU (Daily Total)')
plt.title('Count of Patients in ICU')

In [ ]:
patients_over_time[30], patients_over_time[31], indices_of_time[30]

(12, 11, Timestamp('2011-12-30 00:00:00', freq='D'))

In [ ]:
# Projected Patients in Future
projected_patients = list()
for index_num, date in enumerate(index.index.tolist()):
  str_date = str(date)[0:10]
  #print(index_num, str_date)
  num_patients = len(patient_split[(patient_split['TEST_date_enter_icu'] <= indices_of_time[index_num]) & (patient_split['TEST_date_leave_icu'] >= indices_of_time[index_num]) ])
  patients_over_time.append(num_patients)

print(len(patients_over_time))
patients_over_time[-5:]

plt.rcParams["figure.figsize"]=10,5
plt.plot(indices_of_time[31:61], patients_over_time[31:61],)
plt.xlabel('Date')
plt.ylabel('Patients in ICU (Daily Total)')
plt.title('Count of Patients in ICU')

In [ ]:
count_patients_in_icu = pd.DataFrame({'date': indices_of_time[32:398], 'patients_in_icu': patients_over_time[32:398]})
path = 'mimic_patients_in_icu_' + populate_type + '.feather'
feather.write_dataframe(count_patients_in_icu, path)

In [ ]:
patient_split = patient_split.drop(['first_day_of_hospital','TEST_days_until_icu_entry','TEST_date_diff_days_until_icu_entry'], axis = 1)

# Correlation Results

In [ ]:
index_of_columns = (
    ('time_of_death', 7)
 ,('time_until_death', 7)
    ,('hours_until_death', 8)
    ,('days_until_death',9)
    ,('dischtime',11)
    ,('outtime', 13)
,('time_until_hospital_exit', 14)
 ,('days_until_hospital_exit', 15)
    ,('hours_until_leave_ICU', 16)
    ,('admission_time_until_icu', 17)
     ,('hours_until_icu_admission', 18)
           ,('time_until_leave_ICU', 19)
    ,('hours_until_leave_ICU', 20)
 ,('icu_admit_age', 20)
 ,('female', 21)
 ,('TEST_date_enter_icu', 22)
 ,('TEST_date_leave_icu', 23)
 ,('TEST_date_leave_hospital', 24)
  ,('TEST_date_pass_away', 25)
)
columns_of_df = patient_split.columns.tolist()
for column_name, new_index in index_of_columns:
  #print(column_name, columns_of_df.index(column_name))
  columns_of_df.remove(column_name)
  columns_of_df.insert(new_index, column_name)
#columns_of_df

patient_split = patient_split[columns_of_df]

In [ ]:
patient_split.head(5) # Fields of time_of_death, admittime, intime, gender, first_careunit, hours_until_death

In [ ]:
patient_split.corr()['hours_until_death']

In [ ]:
# Correlations of a little less than 1% to hours until death


import seaborn as sns
cmap = sns.diverging_palette(h_neg=133, h_pos = 10, as_cmap=True)
#data = data.style.background_gradient(cmap=cmap)
#['days_until_death']
#patient_split.corr().iloc[6:,6:].style.background_gradient(cmap=cmap)
plt.style.use('ggplot')
# https://stackoverflow.com/questions/50682797/seaborn-diverging-palette-with-more-than-2-color-tones
with sns.axes_style("white"):
    ax = sns.heatmap(patient_split.corr().iloc[6:,6:], annot=True
                     , fmt='.2f'
    , cmap=cmap
                    , vmin=-1
                     #, vmax=.99
                     , center=0.00
                ,square=True
                , linewidths= 4
                , annot_kws={"size": 8}
                #, cbar_kws={"shrink": .5}
                )
plt.show()

# The collinearity between the related features is around 40% maxinum, so that isn't high enough to cause issues in convergences. If we have issues in convergence, or non-sensical coefficients then features can be removed.
# CSRU: Adult Critical Care - Cardiothoracic
# MICU: Medical Intensive Care Unit

# Can see how there is overlap between the two ICU admission fields, so the less granular of Elective, Emergency, Newborn, Urgent can dropped

In [ ]:
# Absolute Correlation Between All Variables
cm = sns.light_palette('green', as_cmap=True)

s = patient_split.corr().iloc[3:,3:].abs().style.background_gradient(cmap=cm, axis=0)
s

In [ ]:
icd_mask_array = patient_split.corr().columns.str.startswith('icd_')
icd_mask_array[6] = True
icd_mask_array

array([False, False, False, False, False, False,  True, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [ ]:
# Correlation of ICD versus Diagonsis Features

cm = sns.light_palette('red', as_cmap=True)

s = patient_split.corr().loc[:,icd_mask_array].abs().iloc[-17:,:].style.background_gradient(cmap=cm, axis=0)
s

Analysis of Multi-collinearity, 100% of multi-collinearity above .25 is listed below
 
---- High multi-collinearity with definitionally similar entities. Suggest picking one or other based on better fit to training set if lasso fails. One heuristic of better correlation on training set is based upon the correlation in the parenthesis with hours until death (which is a the max date if someone lives)

---- Elevated Multi-collinearity that also has some logical but not definitional connection. Suggest keeping all.

---- Lower multi-collinearity than expected, all other expected areas of multi-collinearity are covered above

In [ ]:
#cm = sns.light_palette('green', as_cmap=True)
cm = sns.diverging_palette(h_neg=133, h_pos = 10, as_cmap=True)

s = patient_split.corr().iloc[3:,3:].style.background_gradient(cmap=cm, axis=0, vmin=-1)
s

# Export to Modeling (ICU LOS + Survival Analysis)

In [ ]:
if proceed_export:
    patient_split.to_csv('static_' + populate_type + '_data.csv')
    print('Exported data of', populate_type)
else:
    print('Not exported, existing files were used instead')

Not exported, existing files were used instead


# Download and Merge ICU Survival Analysis
Note: Only static data when someone leaves the ICU is available, so that is being used as proxy for the ICU Patient View system

In [ ]:
# Uses the day 0 hazard to determine the class, each day 0 hazard is associated with a different day

survival_static_leave_ICU = pd.read_csv('test_curves.csv').drop(['Unnamed: 0'], axis = 1).iloc[:, :367]
survival_static_leave_ICU

In [ ]:
# ICU SURV CURVES
#survival_static = pd.read_csv('icu_surv_curves.csv').drop(['Unnamed: 0'], axis = 1)#.iloc[:, :367]
#survival_static


survival_static = pd.read_csv('unbalanced_icu_surv_curves.csv').drop(['Unnamed: 0'], axis = 1)#.iloc[:, :367]
survival_static

In [ ]:
num_cols = len(survival_static.columns)
print(num_cols)
column_names = list()
column_names.append('subject_id')
for x in range(0, num_cols, 24):
  #print(x)
  column_names.append(str(x))
survival_static = survival_static[column_names]

for x in range(0, num_cols, 24):
    survival_static[str(x)] = survival_static[str(x)].apply(lambda x: 1-x)
    survival_static = survival_static.rename(columns={str(x): str(int(x/24))})
survival_static

In [ ]:
len(survival_static_leave_ICU.merge(survival_static, on = 'subject_id', how = 'inner'))

6541

In [ ]:
merged = patient_split.merge(survival_static, on = 'subject_id', how = 'inner')
print('Total number of shared records', len(merged))

length_forecasts = survival_static.shape[1] - 1
print('Length of forecasts', length_forecasts)

#for num in range (0, length_forecasts):
#  merged[str(num)] = merged[str(num)].apply(lambda x: x)

# *** Used for patients who left ICU
#merged = merged[merged['icu_expire_flag'] == 0 ]
#merged = merged[(merged['TEST_date_leave_icu'] <= current_date_string) & ((merged['TEST_date_pass_away'] >= current_date_string) | (merged['TEST_date_pass_away'].isnull()))]

# *** Used for patients who enter the ICU
current_date_string = '2012-01-01'
merged = merged[(merged['TEST_date_enter_icu'] <= current_date_string) & (merged['TEST_date_leave_icu'] >= current_date_string) & ((merged['TEST_date_pass_away'] >= current_date_string) | (merged['TEST_date_pass_away'].isnull()))]
print('Number of patients', len(merged))

# Even though there is data prior to this point, the current date is the date that the data will be visualized
print('Total number of patients in the ICU on December 1, 2012:', len(merged))
starting_date = convert_year_month_day(pd.to_datetime('2011-12-27', format='%Y-%m-%d', errors = 'coerce'))
print('starting_date:', starting_date)

import datetime as datetime

list_cols_to_forecast = list()

# Loops across all days with forecasts (Takes 1-2 minutes to complete for around 50 records)
for index_day_to_forecast in range(0, length_forecasts): # 365
    #print(index_day_to_forecast)
    day_to_forecast = starting_date + datetime.timedelta(days=index_day_to_forecast)
    #print(day_to_forecast)
    col_day_to_forecast = str(day_to_forecast)[0:10]
    print('col_day_to_forecast:', col_day_to_forecast, day_to_forecast)
    list_cols_to_forecast.append(col_day_to_forecast)
    merged[col_day_to_forecast] = np.nan
    print('index_day_to_forecast', index_day_to_forecast)
    # Loop through rows of patients
    for index, row in merged.iloc[:,].iterrows():

        # look through all previous index dates to look for matches.
        #print('check will run up from ',row['TEST_date_enter_icu'] ,'to', row['TEST_date_enter_icu'] + datetime.timedelta(days=index_day_to_forecast + 2), 'leave icu', row['TEST_date_leave_icu'])
        for previous_index_to_forecast in range(0, index_day_to_forecast + 2):
          # If day to enter the ICU plus index day to forecast is equal to the desired day to forecast, then we must populate a value
          #print('Current row date:', row['TEST_date_enter_icu'] + datetime.timedelta(days=previous_index_to_forecast))
          if row['TEST_date_enter_icu'] + datetime.timedelta(days=previous_index_to_forecast) == day_to_forecast:
              #print('Found a date match!')
              # *** Used if leaving ICU
              #if row['TEST_date_leave_icu'] + datetime.timedelta(days=previous_index_to_forecast) == day_to_forecast:
              value_to_populate = row[str(index_day_to_forecast)]
              merged.loc[merged.index == index, col_day_to_forecast] = value_to_populate
          else:
              #print('Found none')
              # Keeps cell as NAN
              pass

# Delete the integer named columns with hazard functions
str_nums = list()
for num in range (0, length_forecasts):
    str_nums.append(str(num))
merged = merged.drop(str_nums, axis = 1)

merged = merged[~merged[current_date_string].isnull()]

In [ ]:
#The average score on January 1st is 35% comapred to the 3/15 (20%) of patients who passed away soon after January 1st. In this way, the metric possibly seems to suggest the chance that a patient will pass away by that time, when there is a large risk of them passing away on day 1 for many patients.

merged[['subject_id','TEST_date_enter_icu','TEST_date_leave_icu','TEST_date_leave_hospital','TEST_date_pass_away'] + list_cols_to_forecast].sort_values(by='TEST_date_enter_icu')

In [ ]:
merged = merged.sort_values(by=[current_date_string], ascending = False)

def populate_risk_class(x):
    if x > .20:
        return 'high'
    elif x > .10:
        return 'med'
    else:
        return 'low'

merged['risk_class'] = merged[current_date_string].apply(lambda x: populate_risk_class(x))

In [ ]:
merged[[current_date_string,'risk_class']]

# Merge in Static Length of Stay in ICU

In [ ]:
# ICU LOS
import math
los_static = pd.read_csv('static_test_icu_los.csv')[['subject_id','LOS']].rename(columns = {'LOS': 'ICU_LOS'})

los_static['ICU_LOS'] = los_static['ICU_LOS'].apply(lambda x: math.floor(x))

merged = merged.merge(los_static, on = 'subject_id', how = 'inner')

In [ ]:

# Hosp LOS
# los_static = pd.read_csv('static_test_icu_los.csv')[['subject_id','LOS']].rename(columns = {'LOS': 'ICU_LOS'})


# merged['date_forecast_los_hosp_diff'] = merged['HOSP_LOS'].apply(lambda x: datetime.timedelta(days=x)) 
# merged['TEST_date_forecast_los_hosp'] = merged['date_forecast_los_hosp_diff'] + merged['TEST_date_enter_icu'] - merged['admission_time_until_icu']

# merged = merged.merge(los_static, on = 'subject_id', how = 'inner')

# Export to R Shiny Application

In [ ]:
# Prepare data to have columns that can be processed by application
#pd.set_option('display.max_columns', 500)
#merged[['TEST_date_enter_icu','2011-12-01']].head()

In [ ]:
if populate_type == 'test':
  merged_total = patient_split.merge(survival_static, on = 'subject_id', how = 'inner')

  train_diagnosis_statistics = pd.DataFrame()
  for disease in ('cardio','infection','brain','mental','gastro','trauma','respiratory','organ','blood','acute','failure','cancer','back','bone','pain','other'):
      disease_df = merged_total[merged_total[disease] == 1].copy(deep = True)
      #days_until_leave_ICU = disease_df['hours_until_leave_ICU'].mean() / 24
      #average_time_until_death = disease_df['time_until_death'].quantile(0.20)
      enter_average_risk = round(disease_df['0'].mean(),4)
      enter_median_risk = round(disease_df['0'].median(),4)
      week_average_risk = round(disease_df['7'].mean(),3)
      week_median_risk = round(disease_df['7'].median(),3)

      train_diagnosis_statistics = train_diagnosis_statistics.append({'Disease_Group': disease, 'average_risk':enter_average_risk, 'median_risk':enter_median_risk, 'week_average_risk':week_average_risk, 'week_median_risk':week_median_risk}, ignore_index = True)
  if rdata_export:
    train_diagnosis_statistics.to_csv(populate_type + '_survival_stat_by_disease_group.csv')
    print('Refreshed Survival Stats by Disease Group')
  else:
    print('Creating new Survival Stats by Disease Group object')
    display(train_diagnosis_statistics)
else:
    print('No survival data availabe for', populate_type)

In [ ]:
if populate_type == 'train':
  train_diagnosis_statistics = pd.DataFrame()
  for disease in ('cardio','infection','brain','mental','gastro','trauma','respiratory','organ','blood','acute','failure','cancer','back','bone','pain','other'):
      disease_df = patient_split[patient_split[disease] == 1].copy(deep = True)
      days_until_leave_ICU = int(round(disease_df['hours_until_leave_ICU'].mean() / 24, 0))
      train_diagnosis_statistics = train_diagnosis_statistics.append({'Disease_Group': disease
                                                                      , 'Average ICU LOS': days_until_leave_ICU}, ignore_index = True)
  train_diagnosis_statistics['Average ICU LOS'] = train_diagnosis_statistics['Average ICU LOS'].astype(int)
  if rdata_export:
      train_diagnosis_statistics.to_csv('train_disease_group_stats.csv')
      print('Updating train disease group stats')
  else:
      print('Generating new train disease group stats, but not exporting')
else:
  print('Not chaning train disease group stats')
  #test_disease_group_stats = pd.read_csv('test_survival_stat_by_disease_group.csv')


Not chaning train disease group stats


In [ ]:
disease_group_stats = pd.read_csv('test_survival_stat_by_disease_group.csv').merge(pd.read_csv('train_disease_group_stats.csv'), on = 'Disease_Group').drop(['Unnamed: 0_x','Unnamed: 0_y'], axis = 1)

if rdata_export:
    path = 'disease_group_stats.feather'
    feather.write_dataframe(disease_group_stats, path)
    print('Disease group feather updated')
else:
    print('Not changing feather')
disease_group_stats

In [ ]:
rdata_export = True

r_data = merged.copy(deep = True)

r_data[['cardio','infection','brain','mental','gastro','trauma','respiratory','organ','blood','acute','failure','cancer','back','bone','pain','other']]

def create_string(x):
    if x == 1:
        return(', ' + disease)
    else:
        return('')

r_data['Disease_Groups'] = ''
for disease in ('cardio','infection','brain','mental','gastro','trauma','respiratory','organ','blood','acute','failure','cancer','back','bone','pain','other'):
    r_data['Disease_Groups'] = r_data['Disease_Groups'] + r_data[disease].apply(create_string)

def strip_text(x):
    total_len = len(x)
    if total_len > 0:
        return(x[2:])
    else:
        return ''

r_data['Disease_Groups'] = r_data['Disease_Groups'].apply(strip_text)

print('Exporting to Feater')
r_data['gender'] = r_data['female'].apply(lambda x: 'F' if x == 1 else 'M')
r_data['TEST_date_forecast_los_icu'] = r_data['ICU_LOS'].apply(lambda x: datetime.timedelta(days=x)) + convert_year_month_day(pd.to_datetime('2012-01-01', format='%Y-%m-%d', errors = 'coerce')) 

r_data = r_data.drop(['female','time_until_death','admission_time_until_icu','time_until_leave_ICU','time_until_hospital_exit'], axis = 1)

index_of_columns = (
('gender', 21)
,('Disease_Groups',22)
,('ICU_LOS', 23)
,('TEST_date_forecast_los_icu', 24)
,('risk_class', 25)
)
columns_of_df = r_data.columns.tolist()
for column_name, new_index in index_of_columns:
    print(column_name)
    columns_of_df.remove(column_name)
    columns_of_df.insert(new_index, column_name)
r_data = r_data[columns_of_df]


for column in ('dbsource','admission_type','marital_status'):
    r_data[column] = r_data[column].str.title()

for column in ('diagnosis','diagnosis'):
    r_data[column] = r_data[column].str.lower()

if rdata_export:
    path = 'mimic_data_' + populate_type + '.feather'
    feather.write_dataframe(r_data, path)
else:
    print('Not exporting to feather')
    display(r_data)

Exporting to Feater
gender
Disease_Groups
ICU_LOS
TEST_date_forecast_los_icu
risk_class


In [ ]:
#current_date_string

#r_data[['subject_id','risk_class' ,'2011-12-29','2011-12-30','2011-12-31','2012-01-01']]

r_data[['subject_id','risk_class' ,'2011-12-29','2011-12-30','2011-12-31','2012-01-01']]

# EDA

In [ ]:
patient_split['icu_admit_age'].describe()

In [ ]:
patient_split['icu_admit_age'].mode()

0    69
dtype: int64

In [ ]:
# Descriptive

plt.hist(patient_split['icu_admit_age'], 10)

plt.xlabel('Age (Years)')
plt.ylabel('Frequency')
plt.title('Age of Adult Patients is High, Clustering around 70')
plt.grid(True)

plt.show()

In [ ]:
# Formal

plt.hist(patient_split['icu_admit_age'], bins = 10)

plt.xlabel('Age (Years)')
plt.ylabel('Frequency')
plt.title('Age of Adult Patients')
plt.grid(True)

plt.show()

In [ ]:
grouped_df = patient_split[patient_split['anytime_expire_flag'] == 1][['time_until_death','icu_admit_age']]
print(len(grouped_df))
grouped_df['Days_Until_Death'] = grouped_df.time_until_death.astype('timedelta64[D]')

groups = grouped_df.groupby([ pd.cut(grouped_df.icu_admit_age
                                     #, bins = 10 #
                                     , bins=range(10, 100, 10)
)]
                                     )

groups = groups.agg({'Days_Until_Death': ['mean', 'count']})
print(groups['Days_Until_Death']['count'].sum())
groups = groups['Days_Until_Death']['mean']
groups.index = ['11 - 20','21 - 30','31 - 40','41 - 50','51 - 60','61 - 70','71 - 80','81 - 90']

plt.plot(groups)

plt.xlabel('Age Range (Years)')
plt.ylabel('Average Time until Death (Days)')
plt.title('Average Days until Passing Away by Age')
plt.grid(True)

In [ ]:
grouped_df = patient_split[['anytime_expire_flag','icu_admit_age']]
print(len(grouped_df))

groups = grouped_df.groupby([ pd.cut(grouped_df.icu_admit_age
                                     , bins=range(10, 100, 10))])


groups = groups.agg({'anytime_expire_flag': ['mean', 'count']})
print(groups['anytime_expire_flag']['count'].sum())
groups = groups['anytime_expire_flag']['mean']
groups.index = ['11 - 20','21 - 30','31 - 40','41 - 50','51 - 60','61 - 70','71 - 80','81 - 90']


plt.plot(groups)

plt.xlabel('Age Range (Years)')
plt.ylabel('Percentage Pass Away after ICU Entry')
plt.title('Percent Pass Away after ICU Entry by Age')
plt.grid(True)

In [ ]:
patient_split

In [ ]:
plt.hist(patient_split['icu_admit_age'], bins = 10)

plt.xlabel('Age (Years)')
plt.ylabel('Frequency')
plt.title('Age of Adult Patients')
plt.grid(True)

plt.show()

In [ ]:
patient_split

In [ ]:
# Descriptive
plt.hist(patient_split['time_until_death'].dropna().astype('timedelta64[M]') / 12 , bins = 100, density=True, histtype='step',
                           cumulative=True, label='Empirical', color='g')
plt.xlabel('Years until Passing Away')
plt.ylabel('Cumulative Distribution')
plt.title('90% of deaths in 5 years, with spike early on')
plt.axvline(5, color='g')
plt.xticks(np.arange(0, 13, step= 1))
plt.hlines(.90, xmin=0, xmax=12, color='g')
plt.grid(True)

plt.show()

In [ ]:
# Formal
plt.hist(patient_split['time_until_death'].dropna().astype('timedelta64[M]') / 12 , bins = 100, density=True, histtype='step',
                           cumulative=True, label='Empirical', color='g')
plt.axis(xmin=0, xmax=11)
plt.xlabel('Time until Passing Away (Years)')
plt.ylabel('Cumulative Distribution')
plt.title('Cumulative Distribution of Patients Passing Away')
plt.axvline(5, color='g')
plt.xticks(np.arange(0, 12, step= 1))
plt.hlines(.90, xmin=0, xmax=11, color='g')
plt.grid(True)

plt.show()

In [ ]:
# Descriptive
plt.hist(patient_split['time_until_death'].dropna().astype('timedelta64[D]') , bins = 10000, density=True, histtype='step',
                           cumulative=True, label='Empirical')
plt.xlim(0, 90)
plt.xlabel('Days until Passing Away')
plt.ylabel('Cumulative Distribution')
plt.title('Spike in first 70 days, half in first week')
#plt.xticks(np.arange(0, 13, 1))
plt.axvline(7, color='g')
plt.hlines(.20, xmin=0, xmax=90, color='g')
plt.grid(True)

plt.show()

In [ ]:
# Formal
plt.hist(patient_split['time_until_death'].dropna().astype('timedelta64[D]') , bins = 10000, density=True, histtype='step',
                           cumulative=True, label='Empirical')
plt.xlim(0, 90)
plt.xlabel('Time until Passing Away (Days)')
plt.ylabel('Cumulative Distribution')
plt.title('90 Day Cumulative Distribution until Patients Passing Away')
#plt.xticks(np.arange(0, 13, 1))
plt.axvline(7, color='g')
plt.hlines(.20, xmin=0, xmax=90, color='g')
plt.grid(True)

plt.show()

In [ ]:

plt.hist(patient_split[patient_split['icu_expire_flag'] == 1]['time_until_death'].dropna().astype('timedelta64[D]') , bins = 10000, density=True, histtype='step',
                           cumulative=True, label='Empirical')



#plt.hist(patient_split[patient_split['icu_expire_flag'] == 1]['time_until_death'].astype('timedelta64[D]'), 10)
plt.xlabel('Days until Death in ICU')
plt.axvline(14, color='g')
plt.hlines(.90, xmin=0, xmax=100, color='g')
plt.ylabel('Frequency')
plt.title('Almost all ICU Deaths Occur in the first 2 weeks (See 14 Days from ICU Admission)')
plt.grid(True)




In [ ]:
plt.hist(patient_split[patient_split['icu_expire_flag'] == 1]['time_until_death'].dropna().astype('timedelta64[D]') , bins = 10000, density=True, histtype='step',
                           cumulative=True, label='Empirical')
plt.xlim(0, 10)
plt.xlabel('Days until Death in ICU')
plt.ylabel('Frequency')
plt.title('Almost all ICU Deaths Occur in first few days')
plt.grid(True)


In [ ]:
plt.hist(patient_split[(patient_split['anytime_expire_flag'] == 1)]['time_until_death'].astype('timedelta64[h]'), 20000)
plt.xlim(0, 72)
plt.xlabel('Days until Death')
plt.ylabel('Frequency')
plt.title('Time until death drops constantly until 30-48 hours when it levels off')
plt.grid(True)


In [ ]:
plt.hist(patient_split[(patient_split['anytime_expire_flag'] == 1) & (patient_split['hospital_expire_flag'] == 0)]['time_until_death'].astype('timedelta64[h]'), 2000)
#plt.xlim(0, 72)
plt.xlabel('Days until Death')
plt.ylabel('Frequency')
plt.title('Time until death drops constantly until 30-48 hours when it levels off')
plt.grid(True)

In [ ]:
plt.hist(patient_split[(patient_split['anytime_expire_flag'] == 1) & (patient_split['hospital_expire_flag'] == 0)]['time_until_death'].astype('timedelta64[D]'), 200)
plt.xlim(0, 360)
plt.xlabel('Days until Death')
plt.ylabel('Frequency')
plt.title('For those who pass away while at home distribution\nis a longer tail than in hospital patients')
plt.grid(True)

In [ ]:
plt.hist(patient_split[(patient_split['icu_expire_flag'] == 1) & (patient_split['hospital_expire_flag'] == 1)]['time_until_death'].astype('timedelta64[h]'), 400)
plt.xlim(0, 72)
plt.xlabel('Days until Death in ICU')
plt.ylabel('Frequency')
plt.title('Time Until Death in Hospital drops considerably after 24-30 hour mark')
plt.grid(True)


In [ ]:
# data = patient_split[(patient_split['icu_expire_flag'] == 1) & (patient_df['time_until_death'] < pd.Timedelta(48,'h'))]

# data['minutes_until_death'] = data['time_until_death'].astype('timedelta64[m]')

# data

# plt.hist(data['minutes_until_death'] , 10)
# # plt.xlim(0, 24)
# plt.xlabel('Hours')
# plt.ylabel('Cumulative Distribution')
# plt.title('Around 70% of all deaths are before the first year')
# plt.grid(True)

# plt.show()

In [ ]:
patient_split.dtypes

subject_id              int64
hadm_id                 int64
icustay_id              int64
anytime_expire_flag     int64
hospital_expire_flag    int64
                        ...  
back                    int64
bone                    int64
pain                    int64
other                   int64
totalcats               int64
Length: 66, dtype: object